# Composite

Composite is a structural design pattern that lets you compose objects into tree structures and then work with these structures as if they were individual objects.

The parts:
 - Component: Component helps in implementing the default behavior for the interface common to all classes as appropriate. It declares the interface of the objects in the composition and for accessing and managing its child components.
 - Leaf: It defines the behavior for primitive objects in the composition. It represents the leaf object in the composition.
 - Composite: It stores the child component and implements child related operations in the component interface.
 - Client: It is used to manipulate the objects in the composition through the component interface.

## Array backed properties

In [14]:
from dataclasses import dataclass

@dataclass
class Character:
    _power: float
    _health: float
    _intelligence: float

    @property
    def power(self):
        return self.power

    @power.setter
    def power(self, value):
        self.power = value

    # in this implementation, avery time we ad a property, we have to update all methods
    # involving all properties
    # Besides, it is easy to forget sonting here when there are a lots of properties
    def sum(self):
        return self._power + self._health + self._intelligence

    # we have a magic number '3.0' here which depends on the number of properties
    # This is very error-prone.
    def average(self):
        return self.sum() / 3.0

In [22]:
# we optimize the previous class using a dict to manage all properties
# In this implementation, when we add properties, we only add them in the 
# property dict in the init method.
# But we don't need to change other methods such as sum and average,
# which is different from the previous implmentation.

class Character:
    def __init__(self, name, power, health, intel):
        self.name = name
        self.properties = dict(zip(["power", "health", "intelligence"], [power, health, intel]))
    
    @property
    def power(self):
        return self.properties["power"]

    @power.setter
    def power(self, value):
        self.properties["power"] = value

    def sum(self):
        return sum(self.properties.values())

    def average(self):
        return self.sum() / len(self.properties)

In [23]:
character = Character("monster", 2, 9, 4)
print(f"{character.name} has {character.sum()} points in total.")
print(f"{character.name} has {character.average()} points in average.")

monster has 15 points in total.
monster has 5.0 points in average.


## Grouping graph

In [9]:
from abc import abstractmethod
from dataclasses import dataclass

# component
class GraphicObject:
    @abstractmethod
    def draw(self):
        pass
    @abstractmethod
    def move(self, pos):
        pass

# leaf
@dataclass
class Circle(GraphicObject):
    radius: float
    pos: list
    
    def draw(self):
        print(f"Draw a circle of radius {self.radius} at {self.pos}")

    def move(self, pos):
        print(f"Move the circle to {self.pos}")

# leaf
@dataclass
class Rect(GraphicObject):
    pos: list
    size: list

    def draw(self):
        print (f"Draw a rectangle of size {self.size} at {self.pos}")

    def move(self, pos):
        print(f"Move the rectangle to {self.pos}")


# composite
class CompositeObjects(GraphicObject):
    def __init__(self):
        self.objects_list = []

    def draw(self):
        for obj in self.objects_list:
            obj.draw()

    def move(self, pos):
        for obj in self.objects_list:
            obj.move(pos)

    def add(self, obj):
        self.objects_list.append(obj)
        return self

    def remove(self, obj):
        self.objects_list.remove(obj)

In [10]:
leaf1 = Circle(1.0, [1, 1])
leaf2 = Circle(3.0, [5, 1])
leaf3 = Rect([1, 1], [1, 1])
leaf4 = Rect([1, 3], [1, 1])
composites = CompositeObjects()
composites.add(leaf1).add(leaf2).add(leaf3).add(leaf4)

composites.draw()
composites.move([6, 6])


Draw a circle of radius 1.0 at [1, 1]
Draw a circle of radius 3.0 at [5, 1]
Draw a rectangle of size [1, 1] at [1, 1]
Draw a rectangle of size [1, 1] at [1, 3]
Move the circle to [1, 1]
Move the circle to [5, 1]
Move the rectangle to [1, 1]
Move the rectangle to [1, 3]


## Tree Structure

In [1]:
# component interface

import abc

class Storage(abc.ABC):
    @abc.abstractmethod
    def get_size(self):
        raise NotImplementedError
    

In [9]:
# leafs
import random
class File(Storage):
    def __init__(self, name):
        self._name = name
        self._size = random.randint(1, 1000)
        print(f"Created file - {self._name}, size - {self._size} Mb")

    def get_size(self):
        return self._size

In [10]:
# composite

class Folder(Storage):
    def __init__(self, name):
        self._name = name
        self.children = []
        print(f"Created folder - {self._name}")

    def add(self, element: Storage):
        self.children.append(element)

    def remove(self, element):
        self.children.remove(element)

    def get_children(self):
        return self.children

    def get_size(self):
        return sum([s.get_size() for s in self.children])

In [12]:
# usage

root = Folder("my_folder")
sub1 = Folder("level1")
sub2 = Folder("level2")
sub2.add(Folder("level3_1"))
sub2.add(Folder("level3_2"))
sub2.add(File("file1_level3"))
sub2.add(File("file2_level3"))
sub1.add(sub2)
sub1.add(File("file1_level2"))
sub1.add(File("file2_level2"))
root.add(sub1)
root.add(sub2)
print("total size: ", root.get_size())

Created folder - my_folder
Created folder - level1
Created folder - level2
Created folder - level3_1
Created folder - level3_2
Created file - file1_level3, size - 271 Mb
Created file - file2_level3, size - 334 Mb
Created file - file1_level2, size - 735 Mb
Created file - file2_level2, size - 915 Mb
total size:  2860


## Neural Network (more complex tree structure)

In [39]:
# component

class Unit(abc.ABC):
    id = 0

class Neuron(Unit):
    id = 0
    def __init__(self):
        self._in = []
        self._out = []
        self.id += 1
        
    def connect_to(self, other):
        if isinstance(other, Neuron):
            self._out.append(other)
            other._in.append(self)
        elif isinstance(other, NeuronLayer):
            for o in other.nodes:
                self.connect_to(o)
        return other
        
    def get_num_connections(self):
        return len(self._in) + len(self._out)

In [54]:
# composite

class NeuronLayer(Unit):
    def __init__(self, count):
        self.id += 1
        self.nodes = [Neuron() for i in range(count)]

    def connect_to(self, other):
        for n in self.nodes:
            if isinstance(other, Neuron):
                n.connect_to(other)
            if isinstance(other, NeuronLayer):
                for o in other.nodes:
                    n.connect_to(o)
        return other

    def get_num_connections(self):
        return sum(n.get_num_connections() for n in self.nodes)
            

In [55]:
# composite

class Net(Unit):
    def __init__(self, name):
        self._nmae = name
        self.units = []
    def add(self, unit:Unit):
        self.units.append(unit)
        return self
    def remove(self, unit:Unit):
        self.units.remove(unit)
    def get_units(self):
        return self.units
    def get_num_connections(self):
        return sum(n.get_num_connections() for n in self.units)

In [56]:
net = Net("net")
net.add(Neuron()).add(Neuron()).add(NeuronLayer(4)).add(Neuron())
for i, n in enumerate(net.get_units()[:-1]):
    n.connect_to(net.get_units()[i+1])
print(net.get_num_connections())


18
